In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import weights
import json
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

In [19]:
#[difficulty, delta_x, delta_y, hold_weight] IN ->
#[delta_x, delta_y] <- OUT

moves_dataset = []
moves_labels = []
hold_weights = weights.get_weights()

climb_data = {}
with open("../data/climbs.json", 'r') as file:
  climb_data = json.load(file)

In [27]:
for climb in climb_data:
    for i in range(1, len(climb['placements'])-1):
        next = climb['placements'][i+1]
        curr = climb['placements'][i]
        prev = climb['placements'][i-1]
        moves_dataset.append([
            climb['difficulty'],
            curr['x'] - prev['x'],
            curr['y'] - prev['y'],
            hold_weights[curr['x'], curr['y']-1]
        ])
        moves_labels.append([
            next['x'] - curr['x'],
            next['y'] - curr['y']
        ])
dataloader_tmp = []
for i in range(len(moves_dataset)): 
    dataloader_tmp.append((torch.tensor(moves_dataset[i]), torch.tensor(moves_labels[i])))
dataloader = DataLoader(dataloader_tmp, batch_size=16, shuffle=True)

In [29]:

# Define the neural network architecture
class HoldPredictor(nn.Module):
    def __init__(self):
        super(HoldPredictor, self).__init__()
        # Define layers: 
        # - input layer: input_size to hidden_size
        # - hidden layer: hidden_size to hidden_size (optional, can add more layers)
        # - output layer: hidden_size to output_size
        input_size = 4
        hidden_size = 8
        output_size = 2
        
        self.fc1 = nn.Linear(input_size, hidden_size)  # First hidden layer
        self.relu = nn.ReLU()  # ReLU activation
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Output layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the network
        x = self.fc1(x)  # Input to first hidden layer
        x = self.relu(x)  # Apply ReLU activation
        x = self.fc2(x)  # Output from second layer
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Instantiate the model
model = HoldPredictor()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Mean Squared Error loss (for regression)
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Stochastic Gradient Descent optimizer

# Example training loop
num_epochs = 10
losses = []
model.train()
for epoch in range(num_epochs):
    # Example random data (batch of inputs)

    for inputs, outputs in dataloader:

        print(inputs.shape, outputs.shape)

        # Zero gradients from previous step
        optimizer.zero_grad()
    
        # Forward pass
        outputs = model.forward(inputs)
    
        # Compute the loss
        loss = criterion(outputs, labels)
    
        # Backward pass (gradient computation)
        loss.backward()

        losses.append(loss.item())
    
        # Update the weights
        optimizer.step()
    
        # Print the loss every 100 epochs
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
# torch.save(model.state_dict(), 'model.pth')

plt.plot(losses)
plt.show()


torch.Size([16, 4]) torch.Size([16, 2])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported